# Эксперименты с эмбеддерами

In [ ]:
# #!python -m pip install -U numpy --user
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124 --user

In [1]:
%pip install git+https://github.com/huggingface/transformers --user

  Cloning https://github.com/huggingface/transformers to c:\users\никита\appdata\local\temp\pip-req-build-yolbl56v
  Resolved https://github.com/huggingface/transformers to commit 816f4424964c1a1631e303b663fc3d68f731e923
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9991917 sha256=d76f5987ab8ceb269ed8d4368b9283270589e90105ff40b94e94da418db61da7
  Stored in directory: C:\Users\Никита\AppData\Local\Temp\pip-ephem-wheel-cache-pe6nf5ge\wheels\49\a7\50\c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers 'C:\Users\Никита\AppData\Local\Temp\pip-req-build-yolbl56v'


In [2]:
import requests
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F
from torchvision.datasets import CIFAR100, EMNIST, STL10
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
from transformers import (
    AutoTokenizer, AutoModel,
    AutoImageProcessor, ResNetModel,
    CLIPProcessor, CLIPModel,
    AutoModelForCausalLM, BitsAndBytesConfig,
    AutoImageProcessor, SwinModel
)
from urllib.request import urlopen
from huggingface_hub import hf_hub_download
from datasets import load_dataset

from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics

#device = "cuda"
torch_dtype = torch.float16
#torch.set_default_device('cuda')

# Datasets

## CIFAR

In [104]:
# впервые download=True
cifar100 = CIFAR100('../data/cifar', download=True, train=False)
cifar_txt = [f'a photo of {cifar100.classes[img[1]]}' for img in cifar100]
cifar_img = [img[0] for img in cifar100]
cifar_labels = [obj[1] for obj in cifar100]

print(cifar100)
len(cifar100.classes)

100%|██████████| 169001437/169001437 [00:15<00:00, 10653011.78it/s]


Extracting ../data/cifar\cifar-100-python.tar.gz to ../data/cifar
Dataset CIFAR100
    Number of datapoints: 10000
    Root location: ../data/cifar
    Split: Test


100

## EMNIST Digits

In [20]:
emnist = EMNIST('../data/emnist', download=True, train=False, split='digits')
emnist_txt = [f'a picture of number {emnist.classes[img[1]]}' for img in emnist][:10000]
emnist_img = [img[0] for img in emnist][:10000]
emnist_img = [img.convert('RGB') for img in emnist_img]
emnist_labels = [obj[1] for obj in emnist][:10000]

## STL10

In [3]:
stl10 = load_dataset('jxie/stl10', split='test')
stl10_dict = {
    0: 'plane',
    1: 'truck',
    2: 'bird',
    3: 'car',
    4: 'cat',
    5: 'deer',
    6: 'dog',
    7: 'horse',
    8: 'monkey',
    9: 'ship',
}

stl10_img = stl10['image']
stl10_labels = stl10['label']
stl10_txt = [stl10_dict[i] for i in stl10['label']]

## MSCOCO

In [3]:
mscoco = load_dataset('../data/coco', split="test")   # "clip-benchmark/wds_mscoco_captions2017"
#temp_jpg = mscoco['jpg']
mscoco_img = mscoco['jpg'] #[np.array(temp_jpg[i])[:, :, :3] for i in range(5000)]
mscoco_img = [img.convert('RGB') for img in mscoco_img]
mscoco_txt = mscoco['txt']
#ds.save_to_disk('../data/coco')

mscoco

Dataset({
    features: ['__key__', '__url__', 'jpg', 'txt'],
    num_rows: 5000
})

# Models

## CLIP Model
1. Входные эмбеддинги - выбраны
2. Последний слой - не выбраны, чуть хуже 1. и длиннее

In [26]:
def get_clip_embeddings(model_name: str, img_inputs, txt_inputs, batch: int = 1):
    """Получение CLIP эмбеддингов
    
    Args:
        name (str): название модели: "openai/clip-vit-large-patch14", "openai/clip-vit-base-patch32", "../models/CLIP-GmP-ViT-L-14"
    """
    # Проверка на len в img_inputs/txt_inputs
    model = CLIPModel.from_pretrained(model_name)
    processor = CLIPProcessor.from_pretrained(model_name)
    
    for i in range(0, len(txt_inputs), batch):

        inputs = processor(
            text=txt_inputs[i:i+batch],
            images=img_inputs[i:i+batch], 
            return_tensors="pt"
        )
        outputs = model(**inputs)
        
        # Проверка на len в img_inputs/txt_inputs
        if i==0:
            embeds = np.zeros((len(img_inputs), outputs.text_embeds.shape[1]+outputs.image_embeds.shape[1]))
        
        for k in range(0, batch):
            embeds[i+k] = \
                list(outputs.text_embeds[k].detach().cpu().numpy())+\
                list(outputs.image_embeds[k].detach().cpu().numpy())
        
    torch.cuda.empty_cache()
    
    return embeds

## Baseline: ResNet50 + E5_small

In [27]:
def get_resnet50_e5small_embeddings(img_inputs, txt_inputs):
    """Получение resnet50 и e5small эмбеддингов"""
    
    # Инициализация
    def average_pool(last_hidden_states: Tensor,
                     attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    
    resnet_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
    resnet_model = ResNetModel.from_pretrained("microsoft/resnet-50")

    e5_tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
    e5 = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

    
    # Скоринг
    final_embeds = np.zeros((len(img_inputs), 384+2048)) # задал константами, так как baseline

    for i in range(0, len(txt_inputs), 1):
        if i%10 == 0:
            print(i)
        resent_inputs = resnet_processor(img_inputs[i], return_tensors="pt")
        e5_inputs = e5_tokenizer(txt_inputs[i], max_length=512, padding=True, truncation=True, return_tensors='pt')
        
        e5_outputs = e5(**e5_inputs)
        e5_embeddings = average_pool(e5_outputs.last_hidden_state, e5_inputs['attention_mask'])
        e5_embeddings = F.normalize(e5_embeddings, p=2, dim=1)
        
        with torch.no_grad():
            final_embeds[i] = \
                np.concatenate(
                    (e5_embeddings.detach().cpu().numpy(),
                     resnet_model(**resent_inputs).pooler_output.detach().cpu().numpy().reshape(1, 2048)),
                     axis=1
                )
        
        torch.cuda.empty_cache()
    
    return final_embeds

## Swin_transformer и e5-small

In [28]:
def get_swin_e5small_embeddings(img_inputs, txt_inputs):
    """Получение swin_transformer и e5small эмбеддингов"""
    
    # Инициализация
    def average_pool(last_hidden_states: Tensor,
                     attention_mask: Tensor) -> Tensor:
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    
    swin_processor = AutoImageProcessor.from_pretrained("microsoft/swin-base-patch4-window7-224-in22k")
    swin_model = SwinModel.from_pretrained("microsoft/swin-base-patch4-window7-224-in22k")

    e5_tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
    e5 = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

    # Скоринг
    final_embeds = np.zeros((len(img_inputs), 384+1024)) # задал константами, так как baseline

    for i in range(0, len(txt_inputs), 1):
        if i%10 == 0:
            print(i)
        swin_inputs = swin_processor(img_inputs[i], return_tensors="pt")
        e5_inputs = e5_tokenizer(txt_inputs[i], max_length=512, padding=True, truncation=True, return_tensors='pt')
        
        e5_outputs = e5(**e5_inputs)
        e5_embeddings = average_pool(e5_outputs.last_hidden_state, e5_inputs['attention_mask'])
        e5_embeddings = F.normalize(e5_embeddings, p=2, dim=1)
        
        with torch.no_grad():
            final_embeds[i] = \
                np.concatenate(
                    (e5_embeddings.detach().cpu().numpy(),
                     swin_model(**swin_inputs).pooler_output.detach().cpu().numpy()),
                     axis=1
                )
        
        torch.cuda.empty_cache()
    
    return final_embeds

## Кластеризация

In [7]:
def get_clustering_metrics(X, labels, target):
    return [
        # Внутренние меры
        metrics.silhouette_score(X, labels, metric='euclidean'),
        metrics.calinski_harabasz_score(X, labels),
        metrics.davies_bouldin_score(X, labels),
        
        # Внешние меры
        metrics.rand_score(target, labels) if target else 0,
        metrics.normalized_mutual_info_score(target, labels) if target else 0
    ]

def get_clustering_experiment(exp_name: str, img_inputs, txt_inputs,
                              preprocessor_name: str, data_name: str,
                              df_results: pd.DataFrame = None,
                              model_name: str = None, target = None):
    
    if data_name == "cifar":
        kmeans = KMeans(n_clusters=100)
    else:
        kmeans = KMeans(n_clusters=10)
    
    print('exp_name')
    print('preprocessor step')
    if preprocessor_name == "clip":
        embs = get_clip_embeddings(model_name, img_inputs, txt_inputs, batch=1)
    if preprocessor_name == "baseline":
        embs = get_resnet50_e5small_embeddings(img_inputs, txt_inputs)
    if preprocessor_name == "swin_e5":
        embs = get_swin_e5small_embeddings(img_inputs, txt_inputs)
    
    print('preprocessor kmeans step')
    kmeans.fit(embs)
    
    if df_results is None:
        df_results = pd.DataFrame(columns=['method-data', 
                                           'silhouette', 'calinski_harabasz', 'davies_bouldin_score',
                                           'rand_score', 'normalized_mutual_info_score'])
    
    print('calculate metrics step')
    df_results.loc[df_results.shape[0]] = [exp_name]+get_clustering_metrics(embs, kmeans.labels_, target)
    
    
    return df_results

In [18]:
exp_setup = {
    # "clip_l-CIFAR": {
    #     "exp_name": "clip_l-CIFAR",
    #     "preprocessor_name": "clip",
    #     "data_name": "cifar",
    #     "model_name": "openai/clip-vit-large-patch14",
    #     "img_inputs": cifar_img.copy(),
    #     "txt_inputs": cifar_txt.copy(),
    #     "target": cifar_labels,
    # },
    # "clip_b-CIFAR": {
    #     "exp_name": "clip_b-CIFAR",
    #     "preprocessor_name": "clip",
    #     "data_name": "cifar",
    #     "model_name": "openai/clip-vit-base-patch32",
    #     "img_inputs": cifar_img.copy(),
    #     "txt_inputs": cifar_txt.copy(),
    #     "target": cifar_labels,
    # },
    # "baseline-CIFAR": {
    #     "exp_name": "baseline-CIFAR",
    #     "preprocessor_name": "baseline",
    #     "data_name": "cifar",
    #     "img_inputs": cifar_img.copy(),
    #     "txt_inputs": cifar_txt.copy(),
    #     "target": cifar_labels,
    # },
    # "swin_e5-CIFAR": {
    #     "exp_name": "swin_e5-CIFAR",
    #     "preprocessor_name": "swin_e5",
    #     "data_name": "cifar",
    #     "img_inputs": cifar_img.copy(),
    #     "txt_inputs": cifar_txt.copy(),
    #     "target": cifar_labels,
    # },
    
    # "clip_l-MSCOCO": {
    #     "exp_name": "clip_l-MSCOCO",
    #     "preprocessor_name": "clip",
    #     "model_name": "openai/clip-vit-large-patch14",
    #     "data_name": "mscoco",
    #     "img_inputs": mscoco_img.copy(),
    #     "txt_inputs": mscoco_txt.copy(),
    #     "target": None,
    # },
    # "clip_b-MSCOCO": {
    #     "exp_name": "clip_b-MSCOCO",
    #     "preprocessor_name": "clip",
    #     "model_name": "openai/clip-vit-base-patch32",
    #     "data_name": "mscoco",
    #     "img_inputs": mscoco_img.copy(),
    #     "txt_inputs": mscoco_txt.copy(),
    #     "target": None,
    # },
    # "baseline-MSCOCO": {
    #     "exp_name": "baseline-MSCOCO",
    #     "preprocessor_name": "baseline",
    #     "data_name": "mscoco",
    #     "img_inputs": mscoco_img.copy(),
    #     "txt_inputs": mscoco_txt.copy(),
    #     "target": None,
    # },
    # "swin_e5-MSCOCO": {
    #     "exp_name": "swin_e5-MSCOCO",
    #     "preprocessor_name": "swin_e5",
    #     "data_name": "mscoco",
    #     "img_inputs": mscoco_img.copy(),
    #     "txt_inputs": mscoco_txt.copy(),
    #     "target": None,
    # },
    
    "clip_b-STL10": {
        "exp_name": "clip_b-STL10",
        "preprocessor_name": "clip",
        "model_name": "openai/clip-vit-base-patch32",
        "data_name": "stl10",
        "img_inputs": stl10_img.copy(),
        "txt_inputs": stl10_txt.copy(),
        "target": stl10_labels.copy(),
    },
    "baseline-STL10": {
        "exp_name": "baseline-STL10",
        "preprocessor_name": "baseline",
        "data_name": "stl10",
        "img_inputs": stl10_img.copy(),
        "txt_inputs": stl10_txt.copy(),
        "target": stl10_labels.copy(),
    },
    "swin_e5-STL10": {
        "exp_name": "swin_e5-STL10",
        "preprocessor_name": "swin_e5",
        "data_name": "stl10",
        "img_inputs": stl10_img.copy(),
        "txt_inputs": stl10_txt.copy(),
        "target": stl10_labels.copy(),
    },
    "clip_l-STL10": {
        "exp_name": "clip_l-STL10",
        "preprocessor_name": "clip",
        "model_name": "openai/clip-vit-large-patch14",
        "data_name": "stl10",
        "img_inputs": stl10_img.copy(),
        "txt_inputs": stl10_txt.copy(),
        "target": stl10_labels.copy(),
    },
    
    # "clip_b-EMNIST": {
    #     "exp_name": "clip_b-EMNIST",
    #     "preprocessor_name": "clip",
    #     "model_name": "openai/clip-vit-base-patch32",
    #     "data_name": "emnist",
    #     "img_inputs": emnist_img.copy(),
    #     "txt_inputs": emnist_txt.copy(),
    #     "target": emnist_labels,
    # },
    # "baseline-EMNIST": {
    #     "exp_name": "baseline-EMNIST",
    #     "preprocessor_name": "baseline",
    #     "data_name": "emnist",
    #     "img_inputs": emnist_img.copy(),
    #     "txt_inputs": emnist_txt.copy(),
    #     "target": emnist_labels,
    # },
    # "swin_e5-EMNIST": {
    #     "exp_name": "swin_e5-EMNIST",
    #     "preprocessor_name": "swin_e5",
    #     "data_name": "emnist",
    #     "img_inputs": emnist_img.copy(),
    #     "txt_inputs": emnist_txt.copy(),
    #     "target": emnist_labels,
    # },
    # "clip_l-EMNIST": {
    #     "exp_name": "clip_l-EMNIST",
    #     "preprocessor_name": "clip",
    #     "model_name": "openai/clip-vit-large-patch14",
    #     "data_name": "emnist",
    #     "img_inputs": emnist_img.copy(),
    #     "txt_inputs": emnist_txt.copy(),
    #     "target": emnist_labels,
    # },
}

In [19]:
results = None

## Scoring

In [ ]:
for exp in exp_setup.keys():
    print(exp)
    results = get_clustering_experiment(**exp_setup[exp], df_results=results)

4280
4290
4300
4310
4320
4330
4340
4350
4360
4370
4380
4390
4400
4410
4420
4430
4440
4450
4460
4470
4480
4490
4500
4510
4520
4530
4540
4550
4560
4570
4580
4590
4600
4610
4620
4630
4640
4650
4660
4670
4680
4690
4700
4710
4720
4730
4740
4750
4760
4770
4780
4790
4800
4810
4820
4830
4840
4850
4860
4870
4880
4890
4900
4910
4920
4930
4940
4950
4960
4970
4980
4990
5000
5010
5020
5030
5040
5050
5060
5070
5080
5090
5100
5110
5120
5130
5140
5150
5160
5170
5180
5190
5200
5210
5220
5230
5240
5250
5260
5270
5280
5290
5300
5310
5320
5330
5340
5350
5360
5370
5380
5390
5400
5410
5420
5430
5440
5450
5460
5470
5480
5490
5500
5510
5520
5530
5540
5550
5560
5570
5580
5590
5600
5610
5620
5630
5640
5650
5660
5670
5680
5690
5700
5710
5720
5730
5740
5750
5760
5770
5780
5790
5800
5810
5820
5830
5840
5850
5860
5870
5880
5890
5900
5910
5920
5930
5940
5950
5960
5970
5980
5990
6000
6010
6020
6030
6040
6050
6060
6070
6080
6090
6100
6110
6120
6130
6140
6150
6160
6170
6180
6190
6200
6210
6220
6230
6240
6250
6260
6270


In [ ]:
results

In [24]:
results

,method-data,silhouette,calinski_harabasz,davies_bouldin_score,rand_score,normalized_mutual_info_score
0,clip_b-EMNIST,0.164941,960.231015,2.000735,0.956254,0.838490
1,baseline-EMNIST,0.083806,650.689334,2.633190,0.877826,0.490402
2,swin_e5-EMNIST,0.109977,650.949920,2.339482,0.904590,0.580193
3,clip_l-EMNIST,0.289205,1934.123559,1.520208,0.996548,0.983215


In [49]:
results

,method-data,silhouette,calinski_harabasz,davies_bouldin_score,f1_score,rand_score,normalized_mutual_info_score
0,clip_l-CIFAR,0.317251,191.928211,1.474333,0.007421,0.999256,0.993279
1,clip_b-CIFAR,0.255371,163.843154,1.747091,0.000000,0.998281,0.984153
2,baseline-CIFAR,0.031173,48.950943,3.452677,0.004937,0.982901,0.533847
3,swin_e5-CIFAR,0.054847,41.406548,3.329345,0.001234,0.990701,0.804645
4,clip_l-MSCOCO,0.050994,106.564796,3.801835,0.000000,0.000000,0.000000
5,clip_b-MSCOCO,0.063904,131.452354,3.271781,0.000000,0.000000,0.000000
6,baseline-MSCOCO,0.442548,431.290599,3.677789,0.000000,0.000000,0.000000
7,swin_e5-MSCOCO,0.348317,76.934665,3.258689,0.000000,0.000000,0.000000
